In [1]:
import pandas as pd
import openpyxl
from pathlib import Path

data_path = Path('data')
extracted_data = Path('/Users/mauricio/OneDrive - fagundes.info/Projetos/BDE/1.extractedData/aneel/geracao')

# Lendo tabela da Aneel (Siga)

In [21]:
usinas = pd.read_excel(data_path / 'BD SIGA_01_12_2021.xlsx', header=1)

In [3]:
usinas.head()

,Empreendimento,CEG,UF,Fonte,Fase,Origem,Tipo,Tipo de Atuação,Combustível Final,Entrada em Operação,Potência Outorgada (kW),Potência Fiscalizada (kW),Garantia Fisica (kW),Latitude Decimal,Longitude Decimal,Início Vigência,Fim Vigência,Proprietário / Regime de Exploração,Sub-Bacia,Município (s)
0,14 de Julho,UHE.PH.RS.000012-4,RS,UHE,Operação,Hídrica,Potencial hidráulico,Concessão,Potencial hidráulico,25/12/2008,100000.0,100710.0,47500,-29.064753,-51.674942,2001-03-15,2036-03-15,100% para COMPANHIA ENERGÉTICA RIO DAS ANTAS (...,86 - Taquari,"Bento Gonçalves - RS, Cotiporã - RS"
1,29 de Outubro,UTE.PE.SC.029920-0,SC,UTE,Operação,Fóssil,Petróleo,Registro,Óleo Diesel,07/10/2008,62.0,62.0,0,-26.481946,-49.076673,NaT,NaT,100% para Condomínio Residencial 29 de Outubro...,-,Jaraguá do Sul - SC
2,3 Tentos Ijuí,UTE.FL.RS.048608-6,RS,UTE,Construção,Biomassa,Floresta,Autorização,Resíduos Florestais,NaN,5500.0,0.0,0,-28.417778,-53.909722,2021-07-05,2056-07-05,100% para TRÊS TENTOS AGROINDUSTRIAL S.A. (APE),-,Ijuí - RS
3,300KVA SCANIA,UTE.PE.MG.032686-0,MG,UTE,Operação,Fóssil,Petróleo,Registro,Óleo Diesel,01/06/2013,240.0,240.0,0,-21.777436,-46.596899,NaT,NaT,100% para Mineração Curimbaba Ltda. (REG),-,Poços de Caldas - MG
4,3M Itapetininga,UTE.PE.SP.028642-7,SP,UTE,Operação,Fóssil,Petróleo,Registro,Óleo Diesel,12/06/2002,3840.0,3840.0,0,-23.600873,-48.094917,NaT,NaT,100% para 3M do Brasil Ltda (REG),-,Itapetininga - SP


## Análise da tabela de usinas

In [20]:
usinas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11221 entries, 0 to 11220
Data columns (total 20 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Empreendimento                       11221 non-null  object        
 1   CEG                                  11221 non-null  object        
 2   UF                                   11221 non-null  object        
 3   Fonte                                11221 non-null  object        
 4   Fase                                 11221 non-null  object        
 5   Origem                               11221 non-null  object        
 6   Tipo                                 11221 non-null  object        
 7   Tipo de Atuação                      11221 non-null  object        
 8   Combustível Final                    11221 non-null  object        
 9   Entrada em Operação                  9575 non-null   object        
 10  Potência O

Temos 11.221 usinas, sendo que apenas colunas de datas têm nulos, o que faz sentido, uma vez que temos usinas que ainda não entraram em operação. Vamos checar se todas as usinas em operação possuem datas

Como não existe um dicionário de dados, vamos criar um com base no Manual do Usuário, presente no PowerBI oferecido pela Aneel

In [4]:
import importlib
from dados_siga import split_companies
import dados_siga
importlib.reload(dados_siga)

<module 'dados_siga' from '/Users/mauricio/PycharmProjects/siga_aneel/dados_siga.py'>

In [5]:
df = split_companies(usinas)

In [6]:
df.loc[df['participacao'] != 1.0]

,ceg,empresa,regime,participacao
237,UTE.PE.PA.035706-5,GUASCOR DO BRASIL LTDA,PIE,0.790000
238,UTE.PE.PA.035706-5,Soenergy – Sistemas Internacionais de Energia S.A,PIE,0.210000
417,UTE.PE.PA.035708-1,GUASCOR DO BRASIL LTDA,PIE,0.790000
418,UTE.PE.PA.035708-1,Soenergy – Sistemas Internacionais de Energia S.A,PIE,0.210000
494,UHE.PH.MG.027483-6,Aliança Geração de Energia S.A.,PIE,0.873684
...,...,...,...,...
10439,PCH.PH.RO.029526-4,Construtora Centro Minas Ltda,PIE,0.500000
10465,UTE.FL.MA.035557-7,Sergio Luis Salmão Aquino,REG,0.800000
10466,UTE.FL.MA.035557-7,Leonardo Rios Aquino,REG,0.200000
10594,UTE.PE.MT.035390-6,VARZEA GRANDE SHOPPING S.A,REG,0.500000


In [7]:
# df.to_csv(extracted_data / 'empresas.csv')

In [8]:
df.loc[(df['participacao']<1.0) & (df['ceg'].str.startswith('EOL'))]

,ceg,empresa,regime,participacao
9599,EOL.CV.BA.034778-7,EDF EN do Brasil Participações LTDA,PIE,0.9999
9600,EOL.CV.BA.034778-7,PEC Energia S.A.,PIE,0.0001
9615,EOL.CV.BA.049371-6,Maracanã Geração de Energia e Participações S.A.,PIE,0.9900
9616,EOL.CV.BA.049372-4,Maracanã Geração de Energia e Participações S.A.,PIE,0.9900
9617,EOL.CV.BA.049373-2,Maracanã Geração de Energia e Participações S.A.,PIE,0.9900
9618,EOL.CV.BA.049374-0,Maracanã Geração de Energia e Participações S.A.,PIE,0.9900
9619,EOL.CV.BA.049379-1,Maracanã Geração de Energia e Participações S.A.,PIE,0.9900
9620,EOL.CV.BA.049375-9,Maracanã Geração de Energia e Participações S.A.,PIE,0.9900
9621,EOL.CV.BA.049376-7,Maracanã Geração de Energia e Participações S.A.,PIE,0.9900
9622,EOL.CV.BA.049377-5,Maracanã Geração de Energia e Participações S.A.,PIE,0.9900


In [17]:
from dados_siga import separate_ceg, create_types_df
import dados_siga
importlib.reload(dados_siga)

<module 'dados_siga' from '/Users/mauricio/PycharmProjects/siga_aneel/dados_siga.py'>

In [13]:
ceg_list = separate_ceg(usinas)

In [16]:
ceg_list.head()

,ceg,fonte,tipo,uf,nucleo_ceg,dv
0,UHE.PH.RS.000012-4,UHE,Potencial hidráulico,RS,000012,4
1,UTE.PE.SC.029920-0,UTE,Petróleo,SC,029920,0
2,UTE.FL.RS.048608-6,UTE,Floresta,RS,048608,6
3,UTE.PE.MG.032686-0,UTE,Petróleo,MG,032686,0
4,UTE.PE.SP.028642-7,UTE,Petróleo,SP,028642,7


In [18]:
types = create_types_df(usinas)

In [19]:
types

,tipo_id,tipo_ext
0,AI,Agroindustriais
1,BL,Biocombustíveis líquidos
2,CA,Cinética da água
3,CM,Carvão mineral
4,CV,Cinética do vento
5,FL,Floresta
6,GN,Gás natural
7,OF,Outros Fósseis
8,PE,Petróleo
9,PH,Potencial hidráulico
